In [ ]:
import sys
import time
sys.path.insert(0,'../..')
import omama as O

In [ ]:
result_scanner=O.deepsight_result_scanner()

In [ ]:
omama_loader = O.OmamaLoader()
data = O.Data(data_loader=omama_loader, load_cache=True)

In [ ]:
# %load '/home/p.bendiksen001/deephealth/omama/omama/deep_sight/deep_sight_result_scanner.py'
import json
import os
import matplotlib.pyplot as plt
import csv
from collections import Counter
sys.path.insert(0,'../..')
import omama as O

def list_pred_files(filepath, filetype):
    depth_counter = 0
    for root, dirs, files in os.walk(filepath):
        depth_counter += 1
        # print(f"root: {root}; dirs : {dirs} files: {files}")
        if '2d_out' not in root:
            continue
        if 'log.txt' not in files and 'predictions.json' not in files:
            continue
        for name in files:
            # print(f"counter {depth_counter}; name {name}")
            if name.lower().endswith(filetype.lower()):
                json_path = os.path.join(root, name)
                # print(f"2: {json_path}")
                with open(json_path) as file:
                    preds_dict = json.load(file)
                # print(f"preds: {preds_dict}")
                inner_total_counter=0
                inner_error_counter=0
                for key in preds_dict:
                    inner_total_counter+=1
                    updated_tot_count = total_counter[0] + 1
                    total_counter.insert(0, updated_tot_count)
                    total_counter.pop(1)
                    if preds_dict[key]["errors"] != None:
                        aggregate_errors_dict[key] = preds_dict[key]["errors"]
                        inner_error_counter+=1
                        updated_tot_count = total_counter_err[0] + 1
                        total_counter_err.insert(0, updated_tot_count)
                        total_counter_err.pop(1)
                    else:
                        aggregate_preds_dict[key] = {}
                        aggregate_preds_dict[key]["coords"] = preds_dict[key]["coords"]
                        aggregate_preds_dict[key]["score"] = preds_dict[key]["score"]
                print(f"error % for {name} is {inner_error_counter/inner_total_counter}")
                # paths.append(os.path.join(root, file))
            elif name == 'caselist.txt':
                print(os.getcwd())
                with open(os.path.join(root, name), 'r') as fp:
                    line_count=sum(1 for line in fp if line.rstrip())
                    total_line_count=aggregate_num_lines[0]+line_count
                    aggregate_num_lines.insert(0, total_line_count)
                    aggregate_num_lines.pop(1)
                    # print('Total lines:', aggregate_num_lines)  # 8
    error_values = aggregate_errors_dict.values()
    error_values = [item for sublist in error_values for item in sublist]
    error_values = [item.rsplit(':')[0] for item in error_values]
    print(f"total error count : {total_counter_err[0]}")
    print(f"total error %: {total_counter_err[0] / total_counter[0]}")
    # n, bins, patches = plt.hist(error_values)
    # plt.xlabel("Values")
    # plt.ylabel("Frequency")
    # plt.title("Histogram")
    # plt.show()
    len_preds = len(aggregate_preds_dict.keys())
    len_errors = len(aggregate_errors_dict.keys())
    len_preds_dict = {'case_count': len_preds, 'total': len_preds + len_errors }
    len_preds_dict.update(aggregate_preds_dict)
    len_errors_dict = {'case_count': len_errors, 'total': len_errors + len_preds}
    len_errors_dict.update(aggregate_errors_dict)
    # with open('/raid/mpsych/deepsight_run/2D_whitelist_results/2D_preds.txt', 'w') as file:
    #     file.write(json.dumps(aggregate_preds_dict))  # use `json.loads` to do the reverse
    # with open('/raid/mpsych/deepsight_run/2D_whitelist_results/2D_errs.txt', 'w') as file:
    #     file.write(json.dumps(aggregate_errors_dict))  # use `json.loads` to do the reverse
    print(f"total cases: {aggregate_num_lines[0]}")
    return

def get_label_by_sops(sop):
    pass

def Convert(lst):
    new_dict = {}
    for sublist in lst:
        row=sublist
        sop=row[0]
        if sop in aggregate_preds_dict.keys():
            print("ENTERED")
            new_dict[sop] = [aggregate_preds_dict[sop]["score"], row[2]]
    print(f"len heree: {len(new_dict.keys())}")
    return new_dict

def find_matching_rows(filepath, dict_to_match, delimiter=',', quotechar='"'):
    matches = []
    labels = []
    for root, dirs, files in os.walk(filepath):
        for name in files:
            with open(os.path.join(root, name), 'r', newline='') as csvfile:
                reader = csv.reader(csvfile, delimiter=',')
                for row in reader:
                    if row[0] in aggregate_preds_dict.keys():
                        print("Matched Labels SOP with Pred SOP")
                    if row[0] == '2.25.100200417657065059753016354291204569362':
                        print('Matched with Study UID')
                    labels.append(row)
    print(f"Number of Labels cases: {len(labels)}")
    return Convert(labels)

def cross_matirx(dict1, threshold):
    tp=0
    fp=0
    tn=0
    fn=0
    counter=0
    for sop in dict1.keys():
        counter+=1
        if sop_to_pred_label[sop][0] >= threshold and sop_to_pred_label[sop][1] != "NonCancer":
            tp+=1
        if sop_to_pred_label[sop][0] >= threshold and sop_to_pred_label[sop][1] == "NonCancer":
            fp+=1
        if sop_to_pred_label[sop][0] <= threshold and sop_to_pred_label[sop][1] == "NonCancer":
            tn+=1
        if sop_to_pred_label[sop][0] <= threshold and sop_to_pred_label[sop][1] != "NonCancer":
            fn+=1
    print(f"tp: {tp}, fp: {fp}, tn: {tn}, fn: {fn} \nPercentage: tp{tp/counter}, fp: {fp/counter}, tn: {tn/counter}, "
          f"fn: {fn/counter} ")

if __name__ == '__main__':
    paths = []
    aggregate_preds_dict = {}
    aggregate_errors_dict = {}
    num_lines=0
    aggregate_num_lines = [0]
    total_counter = [0]
    total_counter_err = [0]
    list_pred_files('/raid/mpsych/deepsight_run', '.json')
    sop_to_pred_label = find_matching_rows("/raid/data01/deephealth/labels", aggregate_preds_dict)
    print(f"len: {len(sop_to_pred_label)}")
    cross_matirx(sop_to_pred_label, 0.5)

